## Visualizing ICESat-2 dataset from MAAP STAC


**In this notebook, we discover ICESat-2 Boreal COGs from [MAAP's STAC](https://stac.maap-project.org) and generate a mosaic.**

#### MosaicJSON

**A common challenge in visualizing spatial data is data is stored across many files representing small spatial extents.**

MosaicJSON is a specification created by DevelopmentSeed which aims to be an open standard for representing metadata about a spatial mosaic of many COG files.

> MosaicJSON can be seen as a cloud friendly virtual raster (see GDAL's VRT) enabling spatial and temporal indexing for a list of Cloud-Optimized GeoTIFF.

Ref: https://github.com/developmentseed/mosaicjson-spec

#### Titiler Endpoint

By default, TiTiler has a complete `mosaicjson` endpoint. For this demo we are going to use an instance of TiTiler deployed by MAAP at [https://titiler.maap-project.org](https://titiler.maap-project.org)

### Requirements

To be able to run this notebook you'll need the following requirements:

- rasterio
- folium
- requests
- tqdm
- rio-tiler (2.0b8) (Optional)
- cogeo-mosaic (Optional)

In [4]:
# !pip install rasterio folium requests tqdm
# !pip install rio-tiler cogeo-mosaic --pre

### Get the Data

In [60]:
import requests
from pprint import pprint

from rio_tiler.io import COGReader
from rasterio.features import bounds as featureBounds

from folium import Map, TileLayer, GeoJson

##### Fetch ICESat2-Boreal STAC items

In [61]:
stac_endpoint = "https://stac.maap-project.org"

collection = "icesat2-boreal"
stac_json = {
    "collections": [collection],
    "bbox": [4,51.6,14,61.6],
    "limit": 120
}

headers = {
            "Content-Type": "application/json",
            "Accept": "application/geo+json, application/json",
        }

# Read Items

r_stac = requests.post(f"{stac_endpoint}/search", headers=headers, json=stac_json)
items = r_stac.json()
# items['features'][0]

##### Map the data bounds

In [62]:
geojson = {'type': 'FeatureCollection', 'features': items['features']}

bounds = featureBounds(geojson)
zoom_start = 5

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=zoom_start
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)
m

In [42]:
titiler_endpoint = "https://titiler.maap-project.org"  # MAAP titiler endpoint

#### 2. Create Mosaic

We're using the TiTiler deployed by MAAP

##### Create Mosaic

In [35]:
%time
from rio_tiler.io import COGReader

first_cog = geojson['features'][0]['assets']['cog_default']['href']
with COGReader(first_cog) as cog:
    info = cog.info()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


In [37]:
from cogeo_mosaic.mosaic import MosaicJSON

# We are creating the mosaicJSON using the geojson
# SRTMGL1 CODs have a "browse" asset type, so we can create a mosaic of these type="image/tiff" assets
# accesor function provide access to those
mosaicdata = MosaicJSON.from_features(geojson.get('features'), minzoom=zoom_start, maxzoom=info.maxzoom, accessor=lambda feature : feature['assets']['cog_default']['href'])

In [38]:
# Uploading the mosaicjson to the TiTiler
r = requests.post(
    url=f"{titiler_endpoint}/mosaicjson/mosaics",
    headers={
        "Content-Type": "application/vnd.titiler.mosaicjson+json",
    },
    json=mosaicdata.dict(exclude_none=True)).json()

pprint(r)

{'id': 'bfe594a7-bc30-474e-91d4-8ba53fa1423c',
 'links': [{'href': 'https://titiler.maap-project.org/mosaicjson/mosaics/bfe594a7-bc30-474e-91d4-8ba53fa1423c',
            'rel': 'self',
            'title': 'Self',
            'type': 'application/json'},
           {'href': 'https://titiler.maap-project.org/mosaicjson/mosaics/bfe594a7-bc30-474e-91d4-8ba53fa1423c/mosaicjson',
            'rel': 'mosaicjson',
            'title': 'MosiacJSON',
            'type': 'application/json'},
           {'href': 'https://titiler.maap-project.org/mosaicjson/mosaics/bfe594a7-bc30-474e-91d4-8ba53fa1423c/tilejson.json',
            'rel': 'tilejson',
            'title': 'TileJSON',
            'type': 'application/json'},
           {'href': 'https://titiler.maap-project.org/mosaicjson/mosaics/bfe594a7-bc30-474e-91d4-8ba53fa1423c/tiles/{z}/{x}/{y}',
            'rel': 'tiles',
            'title': 'Tiles',
            'type': 'application/json'},
           {'href': 'https://titiler.maap-project.or

The response from the post request gives endpoints for different services (eg. mosaicjson, tilejson, tiles, wmts, etc). We're fetching the `tilejson` endpoint. 

In [39]:
tilejson_endpoint = list(filter(lambda x: x.get("rel") == "tilejson", dict(r)["links"]))

##### Display Tiles

NOTE: You have to zoom to "minzoom" to load the data.

SECOND NOTE: The important bit is the "tiles" endpoint returned from `f"{titiler_endpoint}/mosaicjson/mosaics`. This endpoint (e.g. `https://titiler.maap-project.org/mosaicjson/mosaics/4199126b-9313-435a-b4b5-17802716b7b1/tiles/{z}/{x}/{y}`) could be used in any map client which can tile `x,y,z` layers.

In [40]:
r_te = requests.get(
    tilejson_endpoint[0].get('href')
).json()

pprint(f"{r_te['tiles'][0]}rescale=0,400&colormap_name=gist_earth_r")

tiles = TileLayer(
    tiles=f"{r_te['tiles'][0]}bidx=1&rescale=0,400&colormap_name=gist_earth_r",
    min_zoom=r_te["minzoom"],
    max_zoom=r_te["maxzoom"],
    opacity=1,
    attr="UMD"
)

tiles.add_to(m)
m

'https://titiler.maap-project.org/mosaicjson/mosaics/bfe594a7-bc30-474e-91d4-8ba53fa1423c/tiles/{z}/{x}/{y}@1x?rescale=0,400&colormap_name=gist_earth_r'
